In [1]:
import numpy as np
import pandas as pd

from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import tensorflow as tf
tf.random.set_seed(87)

from keras.models import Model
from keras.layers import Input, Dense ,Dropout 
from keras.layers import BatchNormalization, GaussianNoise, Activation, Concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint

import keras_tuner as kt
from keras_tuner.engine.hyperparameters import HyperParameters
from livelossplot import PlotLossesKeras

import warnings
warnings.filterwarnings('ignore')

# Data preprocessing

1.1. Import data

In [2]:
#############
TICKER = 2330
TP = 10
#############

### import data ###
data = pd.read_csv('/Users/yitsung/Desktop/MasterThesis/data/TaiwanStockData_Top100_EMA')
ticker_data = data[data['ticker']==TICKER].reset_index(drop=True)
ticker_data = ticker_data.drop(columns=['ticker'])

### generate y ###

# ver.1(P-P, 2class) #
# ticker_data[f'y_after_{TP}'] = ticker_data['close'].shift(-TP)
# ticker_data[f'y_after_{TP}'] = ticker_data[f'y_after_{TP}'] - ticker_data['close']
# ticker_data = ticker_data.dropna().reindex()
# ticker_data[f'y_after_{TP}'] = (ticker_data[f'y_after_{TP}'] >= 0).astype(int)

# ver.2(SMA-P/P, 2class) #
ticker_data[f'y_{TP}'] = ticker_data['close'].rolling(window=TP).mean()
ticker_data[f'y_{TP}'] = ticker_data[f'y_{TP}'].shift(-TP)
ticker_data = ticker_data.dropna().reindex()
ticker_data[f'y_{TP}'] = ((ticker_data[f'y_{TP}'] - ticker_data['close']) >= 0).astype(int)

# ver.3(SMA-SMA/SMA, 3class) #
########## not yet ###########

# ### origi data ###
# origi_data = ticker_data.copy()

# ### diff data ###
# ticker_data['open'] = ticker_data['open'].diff()
# ticker_data['high'] = ticker_data['high'].diff()
# ticker_data['low'] = ticker_data['low'].diff()
# ticker_data['close'] = ticker_data['close'].diff()

# ticker_data['EMA9'] = ticker_data['EMA9'].diff()
# ticker_data['EMA12'] = ticker_data['EMA12'].diff()
# ticker_data['EMA26'] = ticker_data['EMA26'].diff()

# ticker_data.replace([float('inf'), -float('inf')], 0, inplace=True) # 不知道為何有些調整過後會變inf, 要拿掉(應該是連兩天的價格都相同)
# ticker_data = ticker_data.dropna().reset_index(drop=True)

# ### move 'y' to the last column ###
# y_column = ticker_data.pop(f'y_after_{TP}')
# ticker_data[f'y_after_{TP}'] = y_column

ticker_data

,Date,open,high,low,close,volume,financing,fi,ii,di,rp,capital,EMA9,EMA12,EMA26,MACD,Signal,RSI14,y_10
0,2021-01-04,530.0,540.0,528.0,536.0,39490.0,454.0,12463.0,-33.0,865.0,2342.0,6.0443,521.295251,518.980386,513.251221,5.729165,3.933239,84.477581,1
1,2021-01-05,536.0,542.0,535.0,542.0,34839.0,-355.0,2884.0,179.0,-451.0,-1374.0,5.3592,525.437881,522.532126,515.535238,6.996887,4.619674,88.417310,1
2,2021-01-06,555.0,555.0,541.0,549.0,55614.0,-256.0,5355.0,105.0,-4163.0,1.0,6.9696,530.151835,526.614084,518.179719,8.434365,5.454306,91.005801,1
3,2021-01-07,554.0,570.0,553.0,565.0,53393.0,2200.0,1671.0,-75.0,2060.0,-402.0,8.7664,537.123278,532.531850,521.861371,10.670478,6.574521,93.325963,1
4,2021-01-08,580.0,580.0,571.0,580.0,62957.0,-502.0,3278.0,187.0,1176.0,-5041.0,9.0658,545.700404,539.847445,526.412277,13.435169,8.026473,94.939847,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
694,2023-11-13,579.0,580.0,571.0,571.0,46556.0,-525.0,16064.0,813.0,109.0,15223.0,9.1131,554.525921,552.006718,546.507936,5.498782,2.554078,64.654070,1
695,2023-11-14,576.0,576.0,571.0,572.0,25367.0,128.0,10421.0,-25.0,1044.0,6574.0,5.2297,558.020737,555.082608,548.396237,6.686370,3.380536,68.577468,1
696,2023-11-15,583.0,583.0,575.0,581.0,53122.0,-749.0,26002.0,-684.0,-71.0,16167.0,7.9897,562.616589,559.069899,550.811331,8.258568,4.356142,78.365103,0
697,2023-11-16,581.0,583.0,578.0,583.0,31059.0,-377.0,12982.0,105.0,518.0,9102.0,6.1995,566.693271,562.751453,553.195677,9.555776,5.396069,84.738876,0


1.2.Splite data into train(Library) and test(Prediction)

In [3]:
Library = ticker_data[ticker_data['Date'] <= '2023-06-30'] # windows=20, 最後預測到6/30
Prediction = ticker_data[(ticker_data['Date'] >= '2023-06-01')&(ticker_data['Date'] <= '2023-10-31')] # windows=20, 從6/1預測7/3開始
Prediction.tail()

,Date,open,high,low,close,volume,financing,fi,ii,di,rp,capital,EMA9,EMA12,EMA26,MACD,Signal,RSI14,y_10
681,2023-10-25,544.0,551.0,544.0,544.0,17137.0,-99.0,-2573.0,651.0,-142.0,1185.0,3.9095,544.649068,543.673118,541.716547,1.956572,0.734048,91.909547,0
682,2023-10-26,530.0,535.0,530.0,531.0,31683.0,487.0,-10712.0,-35173.0,-1744.0,-10261.0,6.9033,541.919255,541.723408,540.922728,0.800679,0.747374,85.178131,1
683,2023-10-27,534.0,536.0,532.0,533.0,17051.0,17.0,-5262.0,1478.0,-73.0,-1739.0,4.1968,540.135404,540.381345,540.335860,0.045485,0.606996,78.927028,1
684,2023-10-30,531.0,534.0,528.0,532.0,23299.0,265.0,-11811.0,487.0,378.0,-5803.0,5.6532,538.508323,539.091907,539.718389,-0.626481,0.360301,72.836426,1
685,2023-10-31,535.0,535.0,527.0,529.0,28073.0,113.0,-9363.0,495.0,-358.0,-5392.0,5.4314,536.606658,537.539306,538.924434,-1.385128,0.011215,63.359478,1


1.3.Data Normalize

In [4]:
def make_data_minmax(Library, Prediction):

    # MinMax #
    scaler_X = MinMaxScaler()
    feature_to_standardize = Library.columns.to_list()[1 : -1]
    Library[feature_to_standardize] = scaler_X.fit_transform(Library[feature_to_standardize])
    Prediction[feature_to_standardize] = scaler_X.fit_transform(Prediction[feature_to_standardize])

    scaler_y = MinMaxScaler()
    Lib_ans = Library.iloc[:, -1].values.reshape(-1, 1)
    Library.iloc[:, -1] = scaler_y.fit_transform(Lib_ans)
    Pred_ans = Prediction.iloc[:, -1].values.reshape(-1, 1)
    Prediction.iloc[:, -1] = scaler_y.fit_transform(Pred_ans)

    return Library, Prediction, scaler_y

### 切train和validation ###
train_Library = Library[: int((len(Library) * 0.8))]
valid_Library = Library[int((len(Library) * 0.8)): ]
train_Library, valid_Library, _ = make_data_minmax(Library=train_Library, Prediction=valid_Library)

### 切完整data ###
Library, Prediction, _ = make_data_minmax(Library=Library, Prediction=Prediction)

1.4.Make window data: X, y

In [5]:
def data_preprocess(data, window_size):

    X = np.array(data.iloc[:, 1: -1])
    y = data.iloc[:, -1].values.reshape(-1, 1)

    data_X, data_y = [], []
    for i in range(len(data) - window_size + 1):
        data_X.append(X[i : (i + window_size), :])
        data_y.append(y[i + window_size - 1])

    data_X, data_y = np.array(data_X), np.array(data_y)
        
    return data_X, data_y

train_X, train_y = data_preprocess(data=train_Library, window_size=20)
valid_X, valid_y = data_preprocess(data=valid_Library, window_size=20)
full_X, full_y = data_preprocess(data=Library, window_size=20)
test_X, test_y = data_preprocess(data=Prediction, window_size=20)

1.4.5.Flatten(MLP only)

In [6]:
def make_X_flatten(X):
    X_flatten = X.reshape((X.shape[0], X.shape[1] * X.shape[2]))

    return X_flatten

train_X = make_X_flatten(train_X)
valid_X = make_X_flatten(valid_X)
full_X = make_X_flatten(full_X)
test_X = make_X_flatten(test_X)

1.5.Over-smapling

In [7]:
ros = RandomOverSampler(random_state=87)
train_X_resampled, train_y_resampled = ros.fit_resample(train_X, train_y)

############
train_y_resampled = train_y_resampled.reshape(-1,1)
############

# 檢查過採樣後的資料大小 #
print("Shape of resampled train_X:", train_X_resampled.shape)
print("Shape of resampled train_y:", train_y_resampled.shape)
# 檢查過採樣後的正類樣本數量 #
print("Number of positive samples after resampling:", train_y_resampled.sum())


Shape of resampled train_X: (514, 340)
Shape of resampled train_y: (514, 1)
Number of positive samples after resampling: 257


In [8]:
ros = RandomOverSampler(random_state=87)
full_X_resampled, full_y_resampled = ros.fit_resample(full_X, full_y)

############
full_y_resampled = full_y_resampled.reshape(-1,1)
############

# 檢查過採樣後的資料大小 #
print("Shape of resampled full_X:", full_X_resampled.shape)
print("Shape of resampled full_y:", full_y_resampled.shape)
# 檢查過採樣後的正類樣本數量 #
print("Number of positive samples after resampling:", full_y_resampled.sum())

Shape of resampled full_X: (618, 340)
Shape of resampled full_y: (618, 1)
Number of positive samples after resampling: 309


# Create model

In [9]:
#############
TUNNING = True

params = {'X_shape': train_X.shape,
          'hidden_units': [160, 176, 144, 16, 80], 
          'dropout_rates': [0.2, 0.0, 0.0, 0.8, 0.0, 0.2, 0.2],
          'ls': 0.001, 'lr': 0.01}
#############

2.1.Create model and find hyperparameter

In [10]:
def tunning_model(hp, X_shape):

    # tf.random.set_seed(87)

    #############################################
    hidden_units = [hp.Int(name=f"units_{i}", min_value=16, max_value=256, step=16) for i in range(1, 6)]
    dropout_rates = [hp.Choice(f"dropout_{i}", [0.0, 0.2, 0.5, 0.8]) for i in range(1, 8)]
    ls = hp.Choice('ls',[1e-2, 1e-3, 1e-5])
    lr = hp.Choice('lr',[1e-2, 1e-3, 1e-5])
    #############################################
    
    inp = Input(shape = (X_shape[1], ))
    x0 = BatchNormalization()(inp)

    encoder = GaussianNoise(dropout_rates[0])(x0)
    encoder = Dense(hidden_units[0])(encoder)
    encoder = BatchNormalization()(encoder)
    encoder = Activation('swish')(encoder)
    
    decoder = Dropout(dropout_rates[1])(encoder)
    decoder = Dense(X_shape[1], name = 'decoder')(decoder)  

    x_ae = Dense(hidden_units[1])(decoder)
    x_ae = BatchNormalization()(x_ae)
    x_ae = Activation('swish')(x_ae)
    x_ae = Dropout(dropout_rates[2])(x_ae)

    out_ae = Dense(1, activation = 'sigmoid', name = 'ae_action')(x_ae)
    
    x = Concatenate()([x0, encoder])
    x = BatchNormalization()(x)
    x = Dropout(dropout_rates[3])(x)

    for i in range(2, len(hidden_units)):
        x = Dense(hidden_units[i])(x)
        x = BatchNormalization()(x)
        x = Activation('swish')(x)
        x = Dropout(dropout_rates[i + 2])(x)
        
    out = Dense(1, activation = 'sigmoid', name = 'action')(x)

    model = tf.keras.models.Model(inputs=inp, outputs=[decoder, out_ae, out])
    model.compile(optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=lr),
                  loss = {'decoder': tf.keras.losses.MeanSquaredError(), 
                          'ae_action': tf.keras.losses.BinaryCrossentropy(label_smoothing=ls),
                          'action': tf.keras.losses.BinaryCrossentropy(label_smoothing=ls), 
                         },
                  metrics = {'decoder': tf.keras.metrics.MeanAbsoluteError(name='MAE'), 
                             'ae_action': tf.keras.metrics.AUC(name='AUC'), 
                             'action': tf.keras.metrics.AUC(name='AUC'), 
                            }, 
                 )
    
    return model

if TUNNING:
    model_fn = lambda hp: tunning_model(hp, X_shape=train_X.shape)
    tuner = kt.BayesianOptimization(model_fn,
                                    objective=kt.Objective("val_action_AUC", direction="max"),
                                    max_trials=10,
                                    executions_per_trial=2,
                                    directory="model_kt",
                                    overwrite=True,
                                    seed=87)
    path = f'model.hdf5'
    ckp = ModelCheckpoint(path, monitor='val_action_AUC', verbose = 0, 
                          save_best_only=True, save_weights_only=True, mode='max')
    es = EarlyStopping(monitor='val_action_AUC', min_delta=1e-4, patience=10, mode='max', 
                       baseline=None, restore_best_weights=True, verbose=1)
    
    tuner.search(train_X, [train_X, train_y, train_y], validation_split=0.2, epochs=100, 
                 batch_size=16, callbacks=[ckp, es], verbose=1)
    model = tuner.get_best_models()[0]

    best_hyperparameters = tuner.get_best_hyperparameters()[0]
    print("Best Hyperparameters:")
    print(best_hyperparameters.values)

Trial 5 Complete [00h 00m 38s]
val_action_AUC: 0.7357352674007416

Best val_action_AUC So Far: 0.8507353067398071
Total elapsed time: 00h 03m 12s

Search: Running Trial #6

Value             |Best Value So Far |Hyperparameter
160               |160               |units_1
176               |48                |units_2
112               |256               |units_3
32                |256               |units_4
144               |240               |units_5
0                 |0.5               |dropout_1
0.5               |0                 |dropout_2
0                 |0                 |dropout_3
0                 |0.2               |dropout_4
0                 |0.8               |dropout_5
0                 |0.8               |dropout_6
0.5               |0.2               |dropout_7
0.01              |0.01              |ls
1e-05             |0.01              |lr

Epoch 1/100
 3/24 [==>...........................] - ETA: 1s - loss: 2.5560 - decoder_loss: 0.8095 - ae_action_loss: 0.8353 -

KeyboardInterrupt: 

2.2.Train model(with parameter)

In [ ]:
def create_model(X_shape, hidden_units, dropout_rates, lr, ls):

    tf.random.set_seed(87)

    inp = Input(shape = (X_shape[1], ))
    x0 = BatchNormalization()(inp)

    encoder = GaussianNoise(dropout_rates[0])(x0)
    encoder = Dense(hidden_units[0])(encoder)
    encoder = BatchNormalization()(encoder)
    encoder = Activation('swish')(encoder)
    
    decoder = Dropout(dropout_rates[1])(encoder)
    decoder = Dense(X_shape[1], name = 'decoder')(decoder)  

    x_ae = Dense(hidden_units[1])(decoder)
    x_ae = BatchNormalization()(x_ae)
    x_ae = Activation('swish')(x_ae)
    x_ae = Dropout(dropout_rates[2])(x_ae)

    out_ae = Dense(1, activation = 'sigmoid', name = 'ae_action')(x_ae)
    
    x = Concatenate()([x0, encoder])
    x = BatchNormalization()(x)
    x = Dropout(dropout_rates[3])(x)

    for i in range(2, len(hidden_units)):
        x = Dense(hidden_units[i])(x)
        x = BatchNormalization()(x)
        x = Activation('swish')(x)
        x = Dropout(dropout_rates[i + 2])(x)
        
    out = Dense(1, activation = 'sigmoid', name = 'action')(x)

    model = tf.keras.models.Model(inputs=inp, outputs=[decoder, out_ae, out])
    model.compile(optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=lr),
                  loss = {'decoder': tf.keras.losses.MeanSquaredError(), 
                          'ae_action': tf.keras.losses.BinaryCrossentropy(label_smoothing=ls),
                          'action': tf.keras.losses.BinaryCrossentropy(label_smoothing=ls), 
                         },
                  metrics = {'decoder': tf.keras.metrics.MeanAbsoluteError(name='MAE'), 
                             'ae_action': tf.keras.metrics.AUC(name='AUC'), 
                             'action': tf.keras.metrics.AUC(name='AUC'), 
                            }, 
                 )

    return model

if TUNNING == False:

    path = f'model.hdf5'
    model = create_model(**params)
    ckp = ModelCheckpoint(path, monitor='val_action_AUC', verbose = 0, 
                          save_best_only=True, save_weights_only=True, mode='max')
    es = EarlyStopping(monitor='val_action_AUC', min_delta=1e-4, patience=10, mode='max', 
                       baseline=None, restore_best_weights=True, verbose=1)
    
    history = model.fit(full_X_resampled, full_y_resampled, # train_X_resampled, train_y_resampled
                        validation_split=0.2, shuffle=True, # validation_data=(valid_X, valid_y),
                        # sample_weight = sw[tr], 
                        epochs=100, batch_size=16, callbacks=[ckp, es], verbose=1)
    
    hist = pd.DataFrame(history.history)
    score = hist['val_action_AUC'].max()
    print(f'AUC:', score)

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Epoch 1/100


2024-03-20 21:54:07.635600: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


31/31 [==============================] - 7s 120ms/step - loss: 2.2125 - decoder_loss: 0.6422 - ae_action_loss: 0.7340 - action_loss: 0.8363 - decoder_MAE: 0.6415 - ae_action_AUC: 0.6418 - action_AUC: 0.4968 - val_loss: 2.1326 - val_decoder_loss: 0.6198 - val_ae_action_loss: 0.8194 - val_action_loss: 0.6934 - val_decoder_MAE: 0.6727 - val_ae_action_AUC: 0.4985 - val_action_AUC: 0.4472
Epoch 2/100
31/31 [==============================] - 1s 35ms/step - loss: 1.8136 - decoder_loss: 0.4464 - ae_action_loss: 0.5731 - action_loss: 0.7941 - decoder_MAE: 0.5429 - ae_action_AUC: 0.7726 - action_AUC: 0.5782 - val_loss: 1.8936 - val_decoder_loss: 0.5517 - val_ae_action_loss: 0.6538 - val_action_loss: 0.6881 - val_decoder_MAE: 0.6337 - val_ae_action_AUC: 0.6659 - val_action_AUC: 0.4466
Epoch 3/100
31/31 [==============================] - 1s 36ms/step - loss: 1.7412 - decoder_loss: 0.3908 - ae_action_loss: 0.5732 - action_loss: 0.7772 - decoder_MAE: 0.5175 - ae_action_AUC: 0.7748 - action_AUC: 0.55

2.3.Test model on one stock

In [ ]:
pred_dir = model.predict(test_X)
pred_dir = pred_dir[2]
pred_dir = (pred_dir > 0.5).astype(int)

result_df = pd.DataFrame(pred_dir, columns=['Pred'])
result_df['True'] = test_y

match_count = (result_df['Pred'] == result_df['True']).sum()
correct = match_count / len(result_df)

print(f'ACC: {correct}\n')
result_df.head(60)

3/3 [==============================] - 1s 83ms/step
ACC: 0.6071428571428571



,Pred,True
0,0,1
1,0,0
2,0,0
3,0,0
4,0,1
5,0,1
6,0,1
7,0,1
8,0,0
9,0,0
